# Première version

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

!pip install torchtext==0.6.0
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

from matplotlib import pyplot as plt


     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 1.2MB 9.4MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [ ]:
"""
Tokenizer et Vocabulaire
"""

!pip3 install spacy

!python3 -m spacy download fr
!python3 -m spacy download de
!python3 -m spacy download en



#src_exts = "de"
src_exts = "fr"
trg_exts = "en"



spacy_src = spacy.load(src_exts)
spacy_trg = spacy.load("en")

def tokenize_src(text):
    return [tok.text for tok in spacy_src.tokenizer(text)]

def tokenize_trg(text):
    return [tok.text for tok in spacy_trg.tokenizer(text)]

source_vocab = Field(tokenize=tokenize_src, lower=True, init_token="<sos>", eos_token="<eos>")
target_vocab = Field(tokenize=tokenize_trg, lower=True, init_token="<sos>", eos_token="<eos>")



     |████████████████████████████████| 14.7MB 8.0MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp37-none-any.whl size=14727027 sha256=840d8eac4d3922d173b507df8706c22b848e37ebc93ad48bd00ede59ccf8a831
  Stored in directory: /tmp/pip-ephem-wheel-cache-8m983r7g/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')
     |████████████████████████████████| 14.9MB 9.0MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=17895b07e3f30284327eb5680a3cf6d6db7f43846f3b10216136752cfce03e40
  Stored in directory: /tmp/pip-ephem-wheel-cache-qa1l42zl/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad56

In [ ]:
"""
Load donnée via le import DE-EN
"""

'''
if src_exts == "fr":
  train_data, valid_data, test_data = Multi30k.splits(
      exts=(".fr", ".en"), fields=(source_vocab, target_vocab)
  )

  source_vocab.build_vocab(train_data, max_size=10000, min_freq=2)
  target_vocab.build_vocab(train_data, max_size=10000, min_freq=2)
'''

'\nif src_exts == "fr":\n  train_data, valid_data, test_data = Multi30k.splits(\n      exts=(".fr", ".en"), fields=(source_vocab, target_vocab)\n  )\n\n  source_vocab.build_vocab(train_data, max_size=10000, min_freq=2)\n  target_vocab.build_vocab(train_data, max_size=10000, min_freq=2)\n'

In [ ]:
"""
Load donnée via le GIT FR-EN
"""


!git clone https://github.com/multi30k/dataset
!pwd
%cd dataset/data/task1/raw
!ls

!gzip  -d val.fr.gz
!gzip  -d train.fr.gz
!gzip  -d test_2016_flickr.fr.gz


!gzip  -d val.en.gz
!gzip  -d train.en.gz
!gzip  -d test_2016_flickr.en.gz




if src_exts == "fr":
  train_src = open('train.fr', encoding='utf-8').read().split('\n')
  val_src = open('val.fr', encoding='utf-8').read().split('\n')
  test_src = open('test_2016_flickr.fr', encoding='utf-8').read().split('\n')

  train_trg = open('train.en', encoding='utf-8').read().split('\n')
  val_trg = open('val.en', encoding='utf-8').read().split('\n')
  test_trg = open('test_2016_flickr.en', encoding='utf-8').read().split('\n')


  
  import pandas as pd

  def data2df(src,trg):
      raw_data = {'Source' : [line for line in src], 'Target': [line for line in trg]}
      df = pd.DataFrame(raw_data, columns=["Source", "Target"])
      # remove very long sentences and sentences where translations are 
      # not of roughly equal length
      df['src_len'] = df['Source'].str.count(' ')
      df['trg_len'] = df['Target'].str.count(' ')
      df = df.query('src_len < 80 & trg_len < 80')
      df = df.query('src_len < trg_len * 1.5  &  trg_len <  src_len * 1.5 ')
      return df 

  train_df = data2df(train_src, train_trg)
  valid_df = data2df(val_src, val_trg)
  test_df = data2df(test_src, test_trg)





  from torch.utils.data.dataset import Dataset

  import torchtext
  source_vocab.is_target  = False
  target_vocab.is_target  = True
    
  class QuackDataset(Dataset):
    def __init__(self, data):
          self.src = data[0]
          self.trg = data[1]
          
          field_dict = {}
          field_dict['src'] = source_vocab
          field_dict['trg'] = target_vocab
          setattr(self, "fields", field_dict)

          data = list(zip(*data))
          self.examples = [None]*len(data)
          for idx, d in enumerate(data):
            ex = torchtext.data.example.Example()
            setattr(ex, "src", d[0])
            setattr(ex, "trg", d[1])
            self.examples[idx] = ex
          
    def __len__(self):
        return len(self.examples)

    #def __getitem__(self, idx):
    #  return self.examples[idx]

    def __iter__(self):
      return QuackIterator(self)

    def __getitem__(self, start, stop=None, step=None):

        if stop == None and step == None:
          idx = start
          return self.examples[idx]

        index = start
        if stop == None:
            end = start + 1
        else:
            end = stop
        if step == None:
            stride = 1
        else:
            stride = step

        x = QuackDataset(self.examples[index:end:stride])
        return x


  class QuackIterator:
    ''' Iterator class '''
    def __init__(self,quackDataset):

        print(type(quackDataset))

        # object reference
        self.quackDataset = quackDataset
        # member variable to keep track of current index
        self._index = 0
    def __next__(self):

        '''Returns the next value from team object's lists '''
        if self._index < len(self.quackDataset.examples):
          result = self.quackDataset[self._index]
          self._index +=1
          return result

        # End of Iteration
        raise StopIteration

  train_list = [[source_vocab.preprocess(item) for item in list(train_df['Source'])],[target_vocab.preprocess(item) for item in list(train_df['Target'])]]
  valid_list = [[source_vocab.preprocess(item) for item in list(valid_df['Source'])],[target_vocab.preprocess(item) for item in list(valid_df['Target'])]]
  test_list = [[source_vocab.preprocess(item) for item in list(test_df['Source'])],[target_vocab.preprocess(item) for item in list(test_df['Target'])] ]

  train_data = QuackDataset(train_list)
  valid_data = QuackDataset(valid_list)
  test_data  = QuackDataset(test_list)


  def flipSourceData(datasets):
    for i,data in enumerate(datasets):
      for j,example in enumerate(data.examples):
        (datasets[i]).examples[j].src = (example.src)[::-1]
    return datasets

  #train_data,valid_data, test_data = flipSourceData(train_data,valid_data, test_data )

  isinstance(train_data, Dataset)

  from collections import Counter, OrderedDict
  def build_vocab_quack(vocab_field,field, *args, **kwargs):
          '''Construct the Vocab object for this field from one or more datasets.

          Arguments:
              Positional arguments: Dataset objects or other iterable data
                  sources from which to construct the Vocab object that
                  represents the set of possible values for this field. If
                  a Dataset object is provided, all columns corresponding
                  to this field are used; individual columns can also be
                  provided directly.
              Remaining keyword arguments: Passed to the constructor of Vocab.
          '''
          counter = Counter()
          sources = []
          for arg in args:
              if isinstance(arg, Dataset):
                  sources += [getattr(arg, field)]


              else:
                  sources.append(arg)
          
          for data in sources:
              for x in data:
                  if not vocab_field.sequential:
                      x = [x]
                  try:
                      counter.update(x)
                  except TypeError:
                      counter.update(chain.from_iterable(x))
          specials = list(OrderedDict.fromkeys(
              tok for tok in [vocab_field.unk_token, vocab_field.pad_token, vocab_field.init_token,
                              vocab_field.eos_token] + kwargs.pop('specials', [])
              if tok is not None))
          vocab_field.vocab = vocab_field.vocab_cls(counter, specials=specials, **kwargs)
          return vocab_field

  source_vocab = build_vocab_quack(source_vocab,'src',train_data, max_size=10000, min_freq=2)
  target_vocab = build_vocab_quack(target_vocab,'trg',train_data, max_size=10000, min_freq=2)




Cloning into 'dataset'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 304 (delta 11), reused 25 (delta 7), pack-reused 262
Receiving objects: 100% (304/304), 18.60 MiB | 31.69 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content
/content/dataset/data/task1/raw
test_2016_flickr.cs.gz	test_2017_mscoco.en.gz	train.en.gz
test_2016_flickr.de.gz	test_2017_mscoco.fr.gz	train.fr.gz
test_2016_flickr.en.gz	test_2018_flickr.cs.gz	val.cs.gz
test_2016_flickr.fr.gz	test_2018_flickr.de.gz	val.de.gz
test_2017_flickr.de.gz	test_2018_flickr.en.gz	val.en.gz
test_2017_flickr.en.gz	test_2018_flickr.fr.gz	val.fr.gz
test_2017_flickr.fr.gz	train.cs.gz
test_2017_mscoco.de.gz	train.de.gz


TypeError: ignored

NameError: ignored

In [ ]:

field_dict = {}
field_dict['src'] = source_vocab
field_dict['trg'] = target_vocab


train_data = Dataset(train_list,field_dict)
valid_data = Dataset(valid_list,field_dict)
test_data  = Dataset(test_list,field_dict)

isinstance(train_data, Dataset)

from collections import Counter, OrderedDict
def build_vocab_quack(vocab_field,field, *args, **kwargs):
        '''Construct the Vocab object for this field from one or more datasets.

        Arguments:
            Positional arguments: Dataset objects or other iterable data
                sources from which to construct the Vocab object that
                represents the set of possible values for this field. If
                a Dataset object is provided, all columns corresponding
                to this field are used; individual columns can also be
                provided directly.
            Remaining keyword arguments: Passed to the constructor of Vocab.
        '''
        counter = Counter()
        sources = []
        for arg in args:
            if isinstance(arg, Dataset):
                sources += [getattr(arg, field)]


            else:
                sources.append(arg)
        
        for data in sources:
            for x in data:
                if not vocab_field.sequential:
                    x = [x]
                try:
                    counter.update(x)
                except TypeError:
                    counter.update(chain.from_iterable(x))
        specials = list(OrderedDict.fromkeys(
            tok for tok in [vocab_field.unk_token, vocab_field.pad_token, vocab_field.init_token,
                            vocab_field.eos_token] + kwargs.pop('specials', [])
            if tok is not None))
        vocab_field.vocab = vocab_field.vocab_cls(counter, specials=specials, **kwargs)
        return vocab_field

source_vocab = build_vocab_quack(source_vocab,'src',train_data, max_size=10000, min_freq=2)
target_vocab = build_vocab_quack(target_vocab,'trg',train_data, max_size=10000, min_freq=2)


TypeError: ignored

In [ ]:
%cd /content

torch.save(source_vocab, "source.pt")
torch.save(target_vocab, "target.pt")

torch.save(train_data.fields, "train_fields.pt")
torch.save(train_data.examples, "train_ex.pt")
torch.save(valid_data.fields, "valid_fields.pt")
torch.save(valid_data.examples, "valid_ex.pt")
torch.save(test_data.fields, "test_fields.pt")
torch.save(test_data.examples, "test_ex.pt")

In [ ]:
% cd /content
!pwd
!ls
!git clone https://github.com/Guyntax/INF8225_Didier
% cd /content/INF8225_Didier

In [ ]:
from torchtext.data import Dataset

reload_source_vocab = torch.load("source.pt")
reload_target_vocab = torch.load("target.pt")

re_train_data_fields = torch.load("train_fields.pt")
re_train_data_examples = torch.load("train_ex.pt")
re_valid_data_fields = torch.load("valid_fields.pt")
re_valid_data_examples = torch.load("valid_ex.pt")
re_test_data_fields = torch.load("test_fields.pt")
re_test_data_examples = torch.load("test_ex.pt")

reload_train_data = Dataset(re_train_data_examples, re_train_data_fields)
reload_valid_data = Dataset(re_valid_data_examples, re_valid_data_fields)
reload_test_data = Dataset(re_test_data_examples, re_test_data_fields)

In [ ]:
"""
def save_examples(dataset, savepath):
    with open(savepath, 'w') as f:

        # Save elements
        for pair in dataset.examples:
            data = [pair.English, pair.French]
            f.write(json.dumps(data))  # Write samples
            f.write("\n")


def load_examples(filename):
    examples = []
    with open(filename, 'r') as f:
        # Read num. elements (not really need it)
        total = json.loads(f.readline())

        # Save elements
        for i in range(total):
            line = f.readline()
            example = json.loads(line)
            # example = data.Example().fromlist(example, fields)  # Create Example obj. (you can do it here or later)
            examples.append(example)

    end = time.time()
    print(end - start)
    return examples


import json
examples = load_examples('train.json')
examples = [data.Example().fromlist(d, data_fields) for d in examples]
train = Dataset(examples, data_fields)

# Reloading the validation dataset
examples = load_examples('val.json')
examples = [data.Example().fromlist(d, data_fields) for d in examples]
val = Dataset(examples, data_fields)

# # Reloading the test dataset
# examples = load_examples(test.json)
# examples = [data.Example().fromlist(d, data_fields) for d in examples]
# test = Dataset(examples, data_fields)
"""